In [1]:
import torch #main module
if not torch.cuda.is_available():
	print("Buy a gpu")
	exit(-1)
import torch.utils.data
import torch.nn as nn
import numpy #numpy module

Buy a gpu


In [3]:
## Specific parameters of network

epochs=10
batch=100
data_len=60000

In [5]:
import os

# Ver el directorio actual

directorio_actual = os.getcwd()
print("Directorio actual:", directorio_actual)

Directorio actual: C:\Users\jazfe\OneDrive\Documentos\CUNEF\redes_neuronales\Redes_neuronales_1\notebook


In [7]:
import os
import urllib.request

# Define la URL y la ruta de destino
url = 'http://www.iro.umontreal.ca/~lisa/deep/data/mnist/mnist.pkl.gz'
destination = 'data/mnist.pkl.gz'

# Crear el directorio si no existe
os.makedirs(os.path.dirname(destination), exist_ok=True)

# Descargar el archivo
urllib.request.urlretrieve(url, destination)

print(f'Archivo descargado y movido a: {destination}')

Archivo descargado y movido a: data/mnist.pkl.gz


In [8]:
import os

# Verifica si el archivo existe en la ruta especificada
if os.path.exists('data/mnist.pkl.gz'):
    print("El archivo se encuentra en la ubicación correcta.")
else:
    print("El archivo no se encuentra en la ruta especificada.")

El archivo se encuentra en la ubicación correcta.


In [13]:
## LOAD MNIST DATASET

#download dataset from Montreal webpage. http://www.iro.umontreal.ca/~lisa/deep/data/mnist/mnist.pkl.gz. Place it in /tmp/

import gzip
import pickle

# Ruta al archivo dentro del directorio actual
dataset = 'data/mnist.pkl.gz'

try:
    with gzip.open(dataset, 'rb') as f:
        try:
            in_set_a, valid_set_a, test_set = pickle.load(f, encoding='latin1')  # Añade 'encoding' si es necesario para Python 3
        except:
            raise NameError("Error loading the dataset")
except FileNotFoundError:
    raise NameError("El archivo no se encuentra en la ruta especificada. Verifica que el archivo 'mnist.pkl.gz' esté en la carpeta 'data'.")
except gzip.BadGzipFile:
    raise NameError("El archivo no está en formato GZIP válido. Intenta volver a descargarlo.")


In [15]:
# Verifica el contenido de los datos cargados
print(f"Train set: {len(in_set_a[0])} samples")
print(f"Validation set: {len(valid_set_a[0])} samples")
print(f"Test set: {len(test_set[0])} samples")

# Ejemplo de acceso a un dato específico
print(f"First training sample: {in_set_a[0][0]}")

Train set: 50000 samples
Validation set: 10000 samples
Test set: 10000 samples
First training sample: [0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0

**Verificación de la Carga de Datos**

**Número de Muestras:**

* Train set: 50000 samples: El conjunto de entrenamiento contiene 50,000 muestras.

* Validation set: 10000 samples: El conjunto de validación contiene 10,000 muestras.

* Test set: 10000 samples: El conjunto de prueba contiene 10,000 muestras.

**Primer Ejemplo de Entrenamiento:**

La salida muestra los valores de los píxeles del primer ejemplo en el conjunto de entrenamiento. Estos valores son números entre 0 y 1, que representan la intensidad de los píxeles en una imagen normalizada del dígito MNIST.

In [22]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# Asigna los datos cargados a las variables adecuadas
train_feat = in_set_a[0]
train_labl = in_set_a[1]

# Prepara tus datos
train_features = torch.tensor(train_feat, dtype=torch.float32)
train_labels = torch.tensor(train_labl, dtype=torch.int64)

In [24]:
# Crear TensorDataset
mnist_dataset_train = TensorDataset(train_features, train_labels)

# Crear DataLoader
train_loader = DataLoader(mnist_dataset_train, batch_size=100, shuffle=True)

In [26]:
'''
Define functions
'''
#Define a function with the typical activations. This activations can be obtained from the nn.Module. 
def activation(x,tip):
	if tip=="sof":
		f=nn.Softmax(dim=1)
		return f(x)
	elif tip=='relu':
		f=nn.ReLU()
		return f(x)
	else:
		print("activation function not present")
		exit(-1)

In [28]:
#forward operation. 
def forward(x):
	x2 = activation(torch.mm(x,w1)+b1,'relu')
	y_pre_activation = torch.mm(x2,w2)+b2
	return y_pre_activation 

In [30]:
#inference. We can use the forward function, however we explicitly apply softmax here though it is not necessary to compute the argmax.
#In the next tutorials I cover when, in my opinion, I like make this difference explictly. A reason could be if you want to directly return the softmax output.
#This is because, in pytorch, the crossentropy loss applies this activation
def inference(x):
	x2 = activation(torch.mm(x,w1)+b1,'relu')
	preactivation =activation( torch.mm(x2,w2)+b2,'sof')
	return preactivation

In [32]:
#loss
CE = nn.CrossEntropyLoss() #this performs softmax plus cros-entropy
def loss(t_,t):
	return CE(t_,t)

In [34]:
#define your layer dimension. We train a fully connected neural network of only one layer with 512 neurons
l1=512
l2=10
input_d=784
target_d=10

In [40]:
import numpy as np

# Crear parámetros de la red (variables compartidas en Theano)
w1 = torch.from_numpy(np.random.normal(0, np.sqrt(2.0 / float(l1)), (input_d, l1)).astype('float32')).requires_grad_()
w2 = torch.from_numpy(np.random.normal(0, np.sqrt(2.0 / float(l2)), (l1, l2)).astype('float32')).requires_grad_()
b1 = torch.zeros((1, l1)).requires_grad_()
b2 = torch.zeros((1, l2)).requires_grad_()
differentiated_params = [w1, w2, b1, b2]

epochs = 20

In [42]:
# Y ahora el bucle principal: forward, cost, backward, update, reset grads
for e in range(epochs):
    ce = 0.0
    for x, t in train_loader:  # Sample one batch
        # No mover a GPU, trabajar en CPU
        predict = forward(x)  # Forward
        o = loss(predict, t)  # Compute loss
        o.backward()  # Compute the gradient which respect to leaves

        ce += o.data.item()  # Store the ce for printing

        for p in differentiated_params:  # Loop over params
            p.data = p.data - 0.01 * p.grad.data  # Gradient descent
            p.grad.data.zero_()  # Reset the gradient

    # Imprimir la pérdida y el error en el conjunto de prueba
    test_features = torch.tensor(test_set[0], dtype=torch.float32)
    test_labels = torch.tensor(test_set[1], dtype=torch.int64)
    test_pred = inference(test_features)  # Compute the softmax from test data
    index = torch.argmax(test_pred, 1).data  # Compute argument maximum
    MC = (index != test_labels).float().mean().item() * 100  # Compute MC error in % | Porcentaje de error
    print(f"Epoch {e} cross entropy {ce / len(train_loader):.5f} and Test error {MC:.3f}")

Epoch 0 cross entropy 0.53617 and Test error 8.720
Epoch 1 cross entropy 0.25945 and Test error 6.770
Epoch 2 cross entropy 0.20718 and Test error 5.930
Epoch 3 cross entropy 0.17654 and Test error 5.430
Epoch 4 cross entropy 0.15505 and Test error 4.930
Epoch 5 cross entropy 0.13880 and Test error 4.650
Epoch 6 cross entropy 0.12601 and Test error 4.470
Epoch 7 cross entropy 0.11529 and Test error 4.290
Epoch 8 cross entropy 0.10661 and Test error 4.080
Epoch 9 cross entropy 0.09950 and Test error 3.880
Epoch 10 cross entropy 0.09284 and Test error 3.840
Epoch 11 cross entropy 0.08718 and Test error 3.820
Epoch 12 cross entropy 0.08226 and Test error 3.590
Epoch 13 cross entropy 0.07772 and Test error 3.560
Epoch 14 cross entropy 0.07358 and Test error 3.560
Epoch 15 cross entropy 0.07002 and Test error 3.350
Epoch 16 cross entropy 0.06684 and Test error 3.360
Epoch 17 cross entropy 0.06386 and Test error 3.270
Epoch 18 cross entropy 0.06116 and Test error 3.250
Epoch 19 cross entropy

**Medidas de Rendimiento**

Las medidas de rendimiento son métricas que se utilizan para evaluar la efectividad de un modelo de aprendizaje automático. Algunas de las medidas más comunes incluyen:

* Loss (Pérdida):

La pérdida es una medida de cuán lejos están las predicciones del modelo de los valores reales. Una pérdida más baja indica que el modelo está haciendo mejores predicciones.

En tu caso, estás utilizando la pérdida de cross entropy (entropía cruzada) para medir cuán bien está funcionando tu modelo en la tarea de clasificación.

* Test Error (Error de Prueba):

El error de prueba es el porcentaje de predicciones incorrectas que realiza el modelo en el conjunto de datos de prueba. Un error más bajo indica que el modelo tiene un mejor rendimiento en datos no vistos durante el entrenamiento.

**Interpretación de los resultados**

* Disminución de la Cross Entropy:

Observa cómo la cross entropy (pérdida) disminuye progresivamente con cada epoch. Esto indica que el modelo está aprendiendo y mejorando su capacidad para hacer predicciones correctas durante el entrenamiento.

Por ejemplo, la pérdida disminuye de 0.53617 en la epoch 0 a 0.05856 en la epoch 19.

* Reducción del Error de Prueba:

El error de prueba también disminuye con cada epoch, lo que sugiere que el modelo está generalizando bien y es capaz de hacer predicciones precisas en datos no vistos.

El error de prueba baja de 8.720% en la epoch 0 a 3.250% en la epoch 19.

**Conclusión**

Mejoras Progresivas: Tanto la pérdida de entropía cruzada como el error de prueba muestran una tendencia descendente, indicando que tu modelo está aprendiendo correctamente y mejorando su rendimiento con cada epoch.

Buen Rendimiento: Al final del entrenamiento, tienes una cross entropy baja y un error de prueba reducido, lo que sugiere que tu modelo está bien entrenado y realiza predicciones precisas.

Estas métricas son fundamentales para evaluar y comparar el rendimiento de tu modelo a lo largo del tiempo y asegurarte de que está aprendiendo y generalizando correctamente.

In [51]:
import gzip 
import pickle 
import numpy as np 
import torch 
import torch.nn as nn 
import torch.optim as optim 
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import KFold

In [53]:
# Definir el número de divisiones para KFold
kf = KFold(n_splits=5)

for train_index, val_index in kf.split(train_feat):
    X_train, X_val = train_feat[train_index], train_feat[val_index]
    y_train, y_val = train_labl[train_index], train_labl[val_index]
    
    # Convertir a tensores de PyTorch
    train_features = torch.tensor(X_train, dtype=torch.float32)
    train_labels = torch.tensor(y_train, dtype=torch.int64)
    val_features = torch.tensor(X_val, dtype=torch.float32)
    val_labels = torch.tensor(y_val, dtype=torch.int64)
    
    # Crear TensorDataset y DataLoader
    train_dataset = TensorDataset(train_features, train_labels)
    val_dataset = TensorDataset(val_features, val_labels)
    train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=100, shuffle=False)
    
    # Reinicializar los parámetros de la red para cada fold
    w1 = torch.from_numpy(np.random.normal(0, np.sqrt(2.0 / float(l1)), (input_d, l1)).astype('float32')).requires_grad_()
    w2 = torch.from_numpy(np.random.normal(0, np.sqrt(2.0 / float(l2)), (l1, l2)).astype('float32')).requires_grad_()
    b1 = torch.zeros((1, l1)).requires_grad_()
    b2 = torch.zeros((1, l2)).requires_grad_()
    differentiated_params = [w1, w2, b1, b2]

    # Entrenamiento
    for e in range(epochs):
        ce = 0.0
        for x, t in train_loader:
            predict = forward(x)  # Forward
            o = loss(predict, t)  # Compute loss
            o.backward()  # Compute the gradient which respect to leaves
            ce += o.data.item()  # Store the ce for printing

            for p in differentiated_params:  # Loop over params
                p.data = p.data - 0.01 * p.grad.data  # Gradient descent
                p.grad.data.zero_()  # Reset the gradient

        # Evaluación en el conjunto de validación
        val_pred = inference(val_features)
        val_index = torch.argmax(val_pred, 1).data
        val_MC = (val_index != val_labels).float().mean().item() * 100  # Compute validation error
        print(f"Fold {kf.get_n_splits()} Epoch {e} cross entropy {ce / len(train_loader):.5f} and Validation error {val_MC:.3f}")

# Evaluar en el conjunto de prueba final
test_pred = inference(test_features)
test_index = torch.argmax(test_pred, 1).data
test_MC = (test_index != test_labels).float().mean().item() * 100  # Compute test error
print(f"Final Test error {test_MC:.3f}")

Fold 5 Epoch 0 cross entropy 0.60262 and Validation error 9.610
Fold 5 Epoch 1 cross entropy 0.29054 and Validation error 7.860
Fold 5 Epoch 2 cross entropy 0.23082 and Validation error 6.960
Fold 5 Epoch 3 cross entropy 0.19653 and Validation error 6.420
Fold 5 Epoch 4 cross entropy 0.17324 and Validation error 5.900
Fold 5 Epoch 5 cross entropy 0.15538 and Validation error 5.290
Fold 5 Epoch 6 cross entropy 0.14139 and Validation error 5.150
Fold 5 Epoch 7 cross entropy 0.12939 and Validation error 4.980
Fold 5 Epoch 8 cross entropy 0.11993 and Validation error 4.800
Fold 5 Epoch 9 cross entropy 0.11149 and Validation error 4.670
Fold 5 Epoch 10 cross entropy 0.10427 and Validation error 4.450
Fold 5 Epoch 11 cross entropy 0.09784 and Validation error 4.310
Fold 5 Epoch 12 cross entropy 0.09229 and Validation error 4.140
Fold 5 Epoch 13 cross entropy 0.08700 and Validation error 4.130
Fold 5 Epoch 14 cross entropy 0.08291 and Validation error 4.000
Fold 5 Epoch 15 cross entropy 0.078

**Interpretación de los Resultados Finales**

Vamos a analizar y entender qué significan estos resultados de entrenamiento y validación:

1. Cross Entropy (Pérdida)
Cross Entropy mide la diferencia entre las distribuciones de probabilidad predicha y verdadera.

Una cross entropy baja significa que tu modelo está prediciendo de manera más precisa.

En las últimas epochs, la cross entropy disminuyó progresivamente, alcanzando 0.06003 en la Epoch 19. Esto indica que el modelo está aprendiendo correctamente y ajustándose bien a los datos de entrenamiento.

2. Validation Error (Error de Validación)
Validation Error es el porcentaje de predicciones incorrectas en el conjunto de datos de validación.

Un validation error bajo indica que el modelo tiene un buen rendimiento en datos no vistos durante el entrenamiento.

Aunque hay algunas fluctuaciones, en general, el validation error muestra una tendencia a la baja, alcanzando 4.250% en la Epoch 19. Esto sugiere que el modelo está mejorando su capacidad de generalización.

3. Final Test Error (Error de Prueba)
Test Error es el porcentaje de predicciones incorrectas en el conjunto de datos de prueba.

Un test error bajo es deseable, ya que indica que el modelo puede hacer predicciones precisas en datos completamente nuevos y no vistos.

Tu modelo logró un Final Test Error de **3.620%**, lo cual es un buen resultado. Esto significa que tu modelo tiene un rendimiento sólido y es capaz de generalizar bien a datos no vistos.

**Conclusión**
* Buen Rendimiento del Modelo: Tanto la cross entropy como el validation error han mostrado una mejora constante a lo largo del entrenamiento, lo que indica que el modelo está aprendiendo y generalizando bien.

* Test Error Bajo: Un Final Test Error de 3.620% sugiere que el modelo está haciendo un buen trabajo al predecir correctamente la mayoría de las muestras en el conjunto de prueba.

* Potencial para Mejoras: Aunque los resultados son buenos, siempre hay espacio para ajustar hiperparámetros, probar diferentes arquitecturas, y utilizar técnicas de regularización para posiblemente mejorar aún más el rendimiento del modelo.